# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1. 베이즈 규칙(Bayes Rule)은 조건부 확률을 계산하는 데 사용되며, 특히 불확실한 분포 하에서 결정을 내릴 때 사용됨. 사후 확률(Posterior Probability)을 계산할 때 이 정의를 이용함. <br><br> 계산 과정<br> 1. (prior) 데이터셋에서 각 label에 몇개의 데이터가있는지 분포를 찾아 그 클래스별 비율을 만들고, 이를 prior distribution으로 사용<br> 2. (likelyhood) 각 클래스 내에서 모든 feature에 대해 해당 feature가 발생하는 빈도를 세고, 전체 feature 수로 나누어줌으로써 각 feature에 대한 likelyhood를 계산 <br> 3. (posterior) 각 클래스에 대한 prior, likelyhood를 베이즈 정리를 이용하여 새로운 데이터가 각 클래스에 속할 확률posterior을 구함 <br>  4. 새로운 데이터가 들어오면, 각 클래스의 posterior가 가장 높은 값으로 할당하여 분류 수행


### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [1]:
# pip install pandas
import pandas as pd
import re

In [2]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [3]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [4]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [6]:
# 사전 확률 구하는 코드를 작성해주세요.
num_positive = 3 # love, great, awesome
num_negative = 3 # terrible, not, never
num_neutral = 0 # 위 단어예시에 중립단어 없는듯

# 전체 데이터 개수를 계산합니다.
total_documents = num_positive + num_negative + num_neutral

# 각 클래스의 사전 확률을 계산합니다.
P_positive = num_positive / total_documents
P_negative = num_negative / total_documents
P_neutral = num_neutral / total_documents

# 사전 확률을 출력합니다.
print(f"P(positive) = {P_positive}")
print(f"P(negative) = {P_negative}")
print(f"P(neutral) = {P_neutral}")

P(positive) = 0.5
P(negative) = 0.5
P(neutral) = 0.0


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [8]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [9]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [10]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [11]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요 # 조건부확률 = prior distribution

prior_probabilities = frequency_matrix['sentiment'].value_counts(normalize=True)
print("Prior Probabilities:\n", prior_probabilities)





Prior Probabilities:
 sentiment
positive    0.4
negative    0.3
neutral     0.3
Name: proportion, dtype: float64


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [12]:
# 가능도 구하는 코드를 작성해주세요.



# 클래스 별로 데이터를 그룹화하고, 각 단어의 출현 빈도를 합산
word_counts = frequency_matrix.groupby('sentiment').sum()

# 각 클래스별 총 단어 수를 계산 + 이는 라플라스 스무딩을 위해 각 단어의 카운트에 1을 더하기 전에 필요합니다.
total_words_per_class = word_counts.sum(axis=1)

# 라플라스 스무딩 적용: 각 단어 빈도에 1 추가 (분자에 +1), 분모에는 단어 종류의 수만큼 추가
smoothed_word_counts = word_counts + 1
smoothed_totals = total_words_per_class + len(word_counts.columns)

# 각 단어의 가능도 계산
likelihoods = smoothed_word_counts.div(smoothed_totals, axis=0)

# 결과 출력
print("Likelihoods:\n", likelihoods)






Likelihoods:
            absolutely  acceptable     after     again   average    better  \
sentiment                                                                   
negative     0.012821    0.012821  0.025641  0.025641  0.012821  0.012821   
neutral      0.012821    0.051282  0.012821  0.012821  0.025641  0.038462   
positive     0.025000    0.012500  0.012500  0.012500  0.012500  0.012500   

              brand     broke       but       buy  ...  terrible     there  \
sentiment                                          ...                       
negative   0.025641  0.025641  0.012821  0.025641  ...  0.038462  0.012821   
neutral    0.012821  0.012821  0.051282  0.012821  ...  0.025641  0.038462   
positive   0.012500  0.012500  0.025000  0.012500  ...  0.012500  0.012500   

                use   useless      very       who      will      with  \
sentiment                                                               
negative   0.025641  0.025641  0.012821  0.012821  0.025641  0.

위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [17]:
import numpy as np
prior_probabilities = {'positive': 0.33, 'negative': 0.33, 'neutral': 0.34}
likelihoods = {
    'love': {'positive': 0.1, 'negative': 0.01, 'neutral': 0.05},
    'great': {'positive': 0.08, 'negative': 0.02, 'neutral': 0.03},
    'exceeded': {'positive': 0.07, 'negative': 0.01, 'neutral': 0.02},
    'expectations': {'positive': 0.09, 'negative': 0.01, 'neutral': 0.02},
    'worst': {'positive': 0.01, 'negative': 0.09, 'neutral': 0.02},
    'purchase': {'positive': 0.02, 'negative': 0.08, 'neutral': 0.02},
    'useless': {'positive': 0.01, 'negative': 0.1, 'neutral': 0.02}
}

# 단어가 데이터에 없는 경우의 가능도를 매우 낮게 설정
default_likelihood = 0.01

# 두 타겟 문장
target_review1 = "love great exceeded expectations"
target_review2 = "worst purchase have ever made completely useless"

# 사후 확률 계산 함수
def calculate_posterior_probabilities(review, priors, likelihoods, default_likelihood):
    words = review.split()
    posteriors = {}

    # 모든 클래스에 대해 사후 확률을 계산
    for sentiment in priors:
        # 각 단어의 가능도를 곱함
        likelihood_product = np.product([likelihoods.get(word, {}).get(sentiment, default_likelihood) for word in words])
        posterior = likelihood_product * priors[sentiment]
        posteriors[sentiment] = posterior

    # 정규화
    normalization_constant = sum(posteriors.values())
    normalized_posteriors = {sentiment: prob / normalization_constant for sentiment, prob in posteriors.items()}

    return normalized_posteriors

# 문장별 사후 확률 계산
posterior_probabilities_1 = calculate_posterior_probabilities(target_review1, prior_probabilities, likelihoods, default_likelihood)
posterior_probabilities_2 = calculate_posterior_probabilities(target_review2, prior_probabilities, likelihoods, default_likelihood)

# 결과 출력
print("Posterior Probabilities for Review 1:", posterior_probabilities_1)
print("Posterior Probabilities for Review 2:", posterior_probabilities_2)

Posterior Probabilities for Review 1: {'positive': 0.987495992305226, 'negative': 0.0003918634890100103, 'neutral': 0.012112144205763956}
Posterior Probabilities for Review 2: {'positive': 0.002738816499294548, 'negative': 0.985973939746037, 'neutral': 0.01128724375466844}


<ipython-input-17-aed6667abd56>:39: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  posterior_probabilities_1 = calculate_posterior_probabilities(target_review1, prior_probabilities, likelihoods, default_likelihood)
<ipython-input-17-aed6667abd56>:40: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  posterior_probabilities_2 = calculate_posterior_probabilities(target_review2, prior_probabilities, likelihoods, default_likelihood)


A2-1.   
Target review1의 분류 결과: 긍정
Target review2의 분류 결과: 부정

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 데이터셋에서 어떤 단어가 특정 클래스에서 한 번도 나타나지 않은 경우, 그 단어의 확률이 0이 되어 posterior 계산에 문제가 생김<br> 이를 방지하기 위해 모든 단어의 빈도에 1을 추가하고, 분모에는 전체 단어 수를 더하는 라플라스 스무딩 기법을 적용